In [1]:
import nltk
import pandas as pd
import spacymoji
import spacy
import preprocessor as tweet_preprocessor
from ttp import ttp
import emoji
import ekphrasis
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [2]:
# Read files
train=pd.read_json("./train.zip")
test=pd.read_json("./test_questions.zip")

In [3]:
# Creating df
train=train[["id","text"]]
test=test[["id","text"]]

In [4]:
train

,id,text
0,1016267329137729536,"@Fhumu_Makhwanya Hi, no you cannot withdraw fu..."
1,1016270646060384256,"@GTalevi Hi, can you please share more informa..."
2,1016272104839434240,"@Mandilele_ Hi, thank you for sharing your exp..."
3,1016275249753743360,@GTalevi Please DM us your Home-Loan account n...
4,1016277410545889281,@Angvanreenen83 Please provide us with it.
...,...,...
96557,1030149995490697217,"@Dopla101 Hi, apologies for the delayed respon..."
96558,1030162359933906945,"@BlackRose__ZA Hi.\nKindly DM your query, affe..."
96559,1030170340473929730,"@aminah_bhengu Hi, please inbox us your ID &am..."
96560,1030172343937716224,"@reitumakwea Hi, please follow &amp; inbox us ..."


In [5]:
test

,id,text
0,1028546931453243392,@_Warith Hi Wari. please contact our Customer ...
1,1028548116297986050,"@ziyaad_seedat Hi there Ziyaad, the Samsung Pa..."
2,1028550157699350528,@180DegreesCC Lovely packaging. How long have ...
3,1028552399307399169,"@ConsolerM Hi there, please give us your conta..."
4,1028559460745150465,@zimcritic Loving yourself looks beautiful on ...
...,...,...
32205,1032883399898423298,"@lee_masilo Hi @lee_masilo, sorry for late res..."
32206,1032886868902334464,@BeckeChris Thanks Chris. Network informed. Ne...
32207,1032887467526049792,@ntandoz_nmz You're welcome
32208,1032888433788825601,"@sne_mthethwa Hi sne_mthethwa, we would like t..."


In [6]:
# Lower case
test['preprocessed'] = test['text'].str.lower()
train['preprocessed'] = train['text'].str.lower()

In [7]:
# Turning text columns to list
train_list = train['preprocessed'].tolist()
test_list = test['preprocessed'].tolist()

In [8]:
# Text preprocessing 
# from https://colab.research.google.com/drive/1Uoog-scrKhLX3QZmrs1ClxCFdGIMH88i?usp=sharing#scrollTo=SV2zCbTag8QR

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/Users/kavinathavarajah/opt/anaconda3/lib/python3.9/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


/Users/kavinathavarajah/opt/anaconda3/lib/python3.9/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [9]:
# Applying text preprocessor
temp_list1 = []

for s in train_list:
    s = (" ".join(text_processor.pre_process_doc(s)))
    temp_list1.append(s)
    
temp_list2 = []

for s in test_list:
    s = (" ".join(text_processor.pre_process_doc(s)))
    temp_list2.append(s)

In [10]:
# Remove punctuation
import re

train_list2 = []
for sentence in temp_list1:
    sentence = re.sub(r"[^\w\d\s\<\>]+",'',sentence)
    train_list2.append(sentence)
    
test_list2 = []
for sentence in temp_list2:
    sentence = re.sub(r"[^\w\d\s\<\>]+",'',sentence)
    test_list2.append(sentence)

In [11]:
# List back to df
train['preprocessed'] = train_list2
test['preprocessed'] = test_list2

In [12]:
# Remove stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')
test['preprocessed'] = test['preprocessed'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
train['preprocessed'] = train['preprocessed'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

#Remove underscore
train['preprocessed'] = train['preprocessed'].replace('_', '', regex=True)
test['preprocessed'] = test['preprocessed'].replace('_', '', regex=True)

In [13]:
# Tokenize
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
train['tokenized']=train['preprocessed'].apply(tt.tokenize)
test['tokenized']=test['preprocessed'].apply(tt.tokenize)

In [14]:
train

,id,text,preprocessed,tokenized
0,1016267329137729536,"@Fhumu_Makhwanya Hi, no you cannot withdraw fu...",<user> hi cannot withdraw funds transfer pures...,"[<user>, hi, cannot, withdraw, funds, transfer..."
1,1016270646060384256,"@GTalevi Hi, can you please share more informa...",<user> hi please share information regarding,"[<user>, hi, please, share, information, regar..."
2,1016272104839434240,"@Mandilele_ Hi, thank you for sharing your exp...",<user> hi thank sharing experience us,"[<user>, hi, thank, sharing, experience, us]"
3,1016275249753743360,@GTalevi Please DM us your Home-Loan account n...,<user> please dm us home loan account number c...,"[<user>, please, dm, us, home, loan, account, ..."
4,1016277410545889281,@Angvanreenen83 Please provide us with it.,<user> please provide us,"[<user>, please, provide, us]"
...,...,...,...,...
96557,1030149995490697217,"@Dopla101 Hi, apologies for the delayed respon...",<user> hi apologies delayed response please re...,"[<user>, hi, apologies, delayed, response, ple..."
96558,1030162359933906945,"@BlackRose__ZA Hi.\nKindly DM your query, affe...",<user> hi kindly dm query affected phone numbe...,"[<user>, hi, kindly, dm, query, affected, phon..."
96559,1030170340473929730,"@aminah_bhengu Hi, please inbox us your ID &am...",<user> hi please inbox us id order number cont...,"[<user>, hi, please, inbox, us, id, order, num..."
96560,1030172343937716224,"@reitumakwea Hi, please follow &amp; inbox us ...",<user> hi please follow inbox us query affecte...,"[<user>, hi, please, follow, inbox, us, query,..."


In [15]:
test

,id,text,preprocessed,tokenized
0,1028546931453243392,@_Warith Hi Wari. please contact our Customer ...,<user> hi wari please contact customer complai...,"[<user>, hi, wari, please, contact, customer, ..."
1,1028548116297986050,"@ziyaad_seedat Hi there Ziyaad, the Samsung Pa...",<user> hi ziyaad samsung pay also work visa ca...,"[<user>, hi, ziyaad, samsung, pay, also, work,..."
2,1028550157699350528,@180DegreesCC Lovely packaging. How long have ...,<user> lovely packaging long catering confecti...,"[<user>, lovely, packaging, long, catering, co..."
3,1028552399307399169,"@ConsolerM Hi there, please give us your conta...",<user> hi please give us contact details assis...,"[<user>, hi, please, give, us, contact, detail..."
4,1028559460745150465,@zimcritic Loving yourself looks beautiful on ...,<user> loving looks beautiful tell us makes pr...,"[<user>, loving, looks, beautiful, tell, us, m..."
...,...,...,...,...
32205,1032883399898423298,"@lee_masilo Hi @lee_masilo, sorry for late res...",<user> hi <user> sorry late response please dm...,"[<user>, hi, <user>, sorry, late, response, pl..."
32206,1032886868902334464,@BeckeChris Thanks Chris. Network informed. Ne...,<user> thanks chris network informed network t...,"[<user>, thanks, chris, network, informed, net..."
32207,1032887467526049792,@ntandoz_nmz You're welcome,<user> welcome,"[<user>, welcome]"
32208,1032888433788825601,"@sne_mthethwa Hi sne_mthethwa, we would like t...",<user> hi snemthethwa would like assist may pl...,"[<user>, hi, snemthethwa, would, like, assist,..."
